In [ ]:
%load_ext autoreload
%autoreload 2

# Ruggedness metric

Questions
1. What is the topological stability of a circuit?
2. What is the ruggedness of each circuit generated by the model?

In [ ]:
import jax.numpy as jnp
import jax
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict


In [ ]:
fn_saves = os.path.join('weight_saves', '01_cvae',
                        # '2024_12_19__16_10_53_saves_hs16_ls32_nlayers2_adam')
                        # 'saves_2025_01_14__16_43_17_prec_sens')
                        # 'saves_2025_01_14__17_44_21_sens')
                        'saves_2025_01_17__16_01_57_sens_no_cat')
# fn_saves = os.path.join('weight_saves', '01_cvae', 'saves_2025_01_09__15_33_42_adapt')
# fn_saves = os.path.join('weight_saves', '01_cvae', 'saves_2025_01_09__21_26_05_prec_sens') 
saves_loaded = load_json_as_dict(fn_saves)
top_write_dir = os.path.join('data', '02_cvae_verify', make_datetime_str())
(
    rng, rng_model, rng_dataset,
    config_norm_x, config_norm_y, config_filter, config_optimisation, config_dataset, config_training, config_model,
    data, x_cols, df,
    x, cond, y, x_train, cond_train, y_train, x_val, cond_val, y_val,
    total_ds, n_batches, BATCH_SIZE, x_datanormaliser, x_methods_preprocessing, y_datanormaliser, y_methods_preprocessing,
    params, encoder, decoder, model, h2mu, h2logvar, reparam
) = init_from_hpos(hpos)

if not os.path.exists(top_write_dir):
    os.makedirs(top_write_dir)

params = saves_loaded[str(list(saves_loaded.keys())[-1])]['params']
params = arrayise(params)

config_bio = load_json_as_dict(config_dataset.filenames_verify_config)
val_config = config_bio['base_configs_ensemble']['generate_species_templates']
val_config.update(config_bio['base_configs_ensemble']['mutation_effect_on_interactions_signal'])

In [ ]:
(
    analytics, ys, ts, y0m, y00s, ts0, fake_circuits, reverse_rates, model_brn, qreactions, ordered_species, input_species, z, sampled_cond
) = verify(params=params,
           rng=rng,
           decoder=decoder,
           df=df,
           cond=np.array([-0.1, 1.1]),
           config_bio=val_config,
           config_norm_y=config_norm_y,
           config_dataset=config_dataset,
           config_model=config_model,
           x_datanormaliser=x_datanormaliser,
           x_methods_preprocessing=x_methods_preprocessing,
           y_datanormaliser=y_datanormaliser,
           output_species=config_dataset.output_species,
           signal_species=config_dataset.signal_species,
           input_species=data[data['sample_name'].notna()]['sample_name'].unique(),
           n_to_sample=int(hpos['eval_n_to_sample']),
           visualise=True,
           top_write_dir=top_write_dir,
           return_relevant=True,
           impose_final_range=(df[get_true_interaction_cols(df, config_dataset.x_type, remove_symmetrical=True, num_species=3)].min().min(),
                               df[get_true_interaction_cols(df, config_dataset.x_type, remove_symmetrical=True, num_species=3)].max().max()))


In [ ]:


data_dir = '../data'
filenames_verify_config = f'{data_dir}/raw/summarise_simulation/2024_11_21_160955/ensemble_config.json',
config_bio = load_json_as_dict(filenames_verify_config)
val_config = config_bio['base_configs_ensemble']['generate_species_templates']
val_config.update(config_bio['base_configs_ensemble']['mutation_effect_on_interactions_signal'])
input_species=data[data['sample_name'].notna()]['sample_name'].unique(),


robustness = calculate_robustness(interactions)


def calculate_robustness(interactions, eps):
    
    perturbations_input = jax.vmap(create_perturbations)(interactions, eps)
    
    perturbations_output = simulate_perturbations(perturbations_input)
    
    robustness = calculate_robustness_from_perturbations(perturbations_output)
    
    
def create_perturbations(interactions, eps):
    
    interactions_expanded = jnp.ones((interactions.shape + 1, interactions.shape)) * interactions
    
    perturbations = interactions_expanded + jnp.eye(*interactions_expanded.shape) * eps
    perturbations_input = jnp.concatenate(interactions, perturbations)
            
    return perturbations_input
    
    
from evoscaper.scripts.verify import setup_model, make_rates, prep_sim, sim
    
def simulate_perturbations(interactions):
    
    model_brn, qreactions, ordered_species, postproc = setup_model(
        interactions, config_bio, input_species)

    forward_rates, reverse_rates = make_rates(
        config_dataset.x_type, interactions, postproc)

    (signal_onehot, signal_target, y00, t0, t1, dt0, dt1, stepsize_controller, save_steps, max_steps, forward_rates, reverse_rates) = prep_sim(
        signal_species, qreactions, interactions, config_bio, forward_rates, reverse_rates)

    analytics, ys, ts, y0m, y00s, ts0 = sim(y00, forward_rates[0], reverse_rates,
                                            qreactions,
                                            signal_onehot, signal_target,
                                            t0, t1, dt0, dt1,
                                            save_steps, max_steps,
                                            stepsize_controller)
    
    
def calculate_robustness_from_perturbations(perturbations_output):
    
    robustness = jnp.sqrt(jnp.sum(jnp.square(perturbations_output)))
            
    return robustness